--------------------------------------------------

See annotated [K20P64M72SF1RM][1] manual (MK20DX128 processor used on [Teensy 3.1/3.2][2]).

Functional description of Programmable Delay Block (PDB) on page 761.

[1]: https://xodo.com/app/d/?/=9efbee54-ee85-4772-8e5c-859fcb213d74
[2]: https://www.pjrc.com/teensy/

--------------------------------------------------

# PDBx_SC: Status and Control Register

Page 753.

 - `LDMOD`: Load mode select (2 bits)
     * 0: immediate, 1: after counter reaches MOD value, 2: trigger input event, 3: either 1 or 2
 - `PDBEIE`: PDB Sequence error interrupt enable
 - `SWTRIG`: Software trigger, reset and restart counter if software trigger set as input
 - `DMAEN`: DMA enable. 1: `PDBIF` flag generates a DMA request instead of an interrupt.
 - `PRESCALER`: Prescaler divider select (3 bits)
     * 0: /MULT, 1: /2MULT, 2: /4MULT, 3: /8MULT, 4: /16MULT, 5: /32MULT, 6: /64MULT, 7: /128MULT
 - `TRGSEL`: Trigger input select (4 bits)
     * 0-14: trigger-in x, 15: software trigger **(enable `SWTRIG`)**
 - `PDBEN`: PDB enable
 - `PDBIF`: PDB interrupt flag, set when counter value is equal to the `IDLY` register. Writing zero clears this bit.
 - `PDBIE`: PDB interrupt enable, when this bit is set and `DMAEN` is cleared, `PDBIF` generates a PDB interrupt.
 - `MULT`: Multiplication Factor Select for Prescaler (2 bits), `MULT` for `PRESCALER`
     * 0: 1, 1: 10, 2: 20, 3: 40
 - `CONT`: Continuous mode enable
 - `LDOK`: Load OK, update internal registers of `MOD`, `IDLY`, `CHnDLYm`, `DACINTx`, and `POyDLY` according to `LDMOD` scheme.
 
 
## Teensy ADC_Module ##

 - `TRGSEL` = 15 (Software trigger)
 - `PDBEN` = 1
 - `PDBIE` = 1
 - `CONT` = 1
 - `LDMOD` = 0
 - `PRESCALER` = based on frequency
 - `MULT` = based on frequency
 
Use `SWTRIG` to start first count.  Counter continues to loop due to continuous mode (`CONT=1`).

--------------------------------------------------

# PDBx_MOD: Modulus Register

Page 755.

 - `MOD`: PDB modulus (16 bits), specifies the period of the counter.
 
 
## Teensy ADC_Module ##

 - `MOD` = based on frequency
 
Set to `(F_BUS / frequency) - 1`, unless `(F_BUS / frequency)` cannot fit in 16 bits.
In that case, first use `SC[MULT]` and `SC[PRESCALER]` to get close to the requested
period, then further divide `(F_BUS / frequency)` accordingly.

--------------------------------------------------

# PDBx_CNT: Counter Register

Page 756.

 - `CNT`: PDB Counter (16 bits), current value of the counter

--------------------------------------------------

# PDBx_IDLY: Interrupt Delay Register

Page 756.

 - `IDLY`: PDB Interrupt delay (16 bits), delay value to schedule the PDB interrupt.
     * Can be used to schedule an independent interrupt at some point in the PDB cycle.
     * If enabled, a PDB interrupt is generated when the counter is equal to the `IDLY`
     
     
## Teensy ADC_Module ##

 - `IDLY = 1`: i.e., the pdb interrupt when `CNT = 1`.


--------------------------------------------------

# PDBx_CHnC1: Channel n Control Register 1

Page 757.  One bit per channel.

**TODO How manny PDB channels on Teensy 3.1 controller?**

 - `BB`: PDB Channel Pre-Trigger Back-to-Back Operation Enable (8 bits)
     * ??
 - `TOS`: PDB Channel Pre-Trigger Output Select (8 bits)
     * 0: Bypassed mode. Pre-trigger asserts one clock cycle after rising edge on trigger input.
     * 1: Pre-trigger asserts when counter reaches delay `IDLY` *and* one clock cycle after
       rising edge on trigger input.
 - `EN`: PDB Channel Pre-Trigger Enable (8 bits)
     * **TODO Does each bit correspond to an ADC channel?**
 
 
## Teensy ADC_Module ##

Assert pre-trigger when counter reaches `IDLY` (1) and after trigger (software in this case).

 - `TOS = 1`
 - `EN = 1`

--------------------------------------------------

# PDBx_CHnS: Channel n Status Register

Page 758.  One bit per channel.

 - `CF`: PDB Channel Flags, the `CF[m]` bit is set when the PDB counter matches the `CHnDLYm`
 - `ERR`: PDB Channel Sequence Error Flags, error bit is set if a new conversion is triggered during a conversion

--------------------------------------------------

# PDBx_CHnDLY0: Channel n Delay 0 Register

Page 758.

 - `DLY`: PDB Channel delay,  pre-trigger asserts when the counter is equal to `DLY`. (16 bits)

--------------------------------------------------

# PDBx_CHnDLY1: Channel n Delay 1 Register

Page 759.

 - `DLY`: PDB Channel delay,  pre-trigger asserts when the counter is equal to `DLY`. (16 bits)

--------------------------------------------------

# PDBx_DACINTCn: DAC Interval Trigger n Control Register

Page 759.

 - `EXT`: DAC External Trigger Input Enable
     * 0: DAC interval counter reset and started after rising edge on trigger input.
     * 1: DAC interval counter is bypassed, DAC external trigger input triggers the DAC interval trigger.
 - `TOE`: DAC Interval Trigger Enable

--------------------------------------------------

# PDBx_DACINTn: DAC Interval n Register

Page 760.

 - `INT`: DAC interval, triggers `DAC[1:0]` update when the DAC interval counter is equal to the `DACINT`.

--------------------------------------------------

# PDBx_POEN: Pulse-Out n Enable Register (Page 760)

 - `POEN`: PDB Pulse-out enable

--------------------------------------------------

# PDBx_POnDLY: Pulse-Out n Delay Register (Page 761)

 - `DLY1`: Pulse-out delay 1, pulse-out goes high when the PDB counter is equal to the `DLY1`.
 - `DLY2`: Pulse-out delay 2, pulse-out goes low when the PDB counter is equal to the `DLY2`.